**Note:** this notebook was used to create the meta-dataset for the workshop submission. Not needed for later work.

In [112]:
import pandas as pd
import numpy as np

TIME_FORMAT = "%Y%m%d_%H%M%S"

df = pd.read_csv("/Users/zzzzzz/research/active_projects/reczilla/results/results.csv", sep=";")

/Users/zzzzzz/miniconda3/envs/recsys/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (13,14,16,17,31,32,38,40,41,42,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [113]:
# slice by experiment name
df_expt = df.loc[df["experiment_name"].str.startswith("full-experiment-"), :]

df_expt.head()

,alg_name,alg_seed,cutoff_list,dataset_name,exception,experiment_name,hyperparameters_source,num_samples,original_split_path,param_alpha,...,test_metric_USERS_IN_GT_cut_5,test_metric_USERS_IN_GT_cut_50,test_metric_USERS_IN_GT_cut_6,test_metric_USERS_IN_GT_cut_7,test_metric_USERS_IN_GT_cut_8,test_metric_USERS_IN_GT_cut_9,time,time_on_test,time_on_train,time_on_val
4,UserKNNCF_asymmetric,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40...",MovieTweetingsReader,NaN,full-experiment-knn-146,default,100,gs://reczilla-results/dataset-splits/splits-v3...,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,20220417_233953,114.638383,11.768501,115.430055
5,UserKNNCF_asymmetric,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40...",MovieTweetingsReader,NaN,full-experiment-knn-146,random_0,100,gs://reczilla-results/dataset-splits/splits-v3...,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,20220417_233953,124.220636,13.444535,123.385043
6,UserKNNCF_asymmetric,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40...",MovieTweetingsReader,NaN,full-experiment-knn-146,random_1,100,gs://reczilla-results/dataset-splits/splits-v3...,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,20220417_233953,115.785270,11.932769,115.417125
7,UserKNNCF_asymmetric,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40...",MovieTweetingsReader,NaN,full-experiment-knn-146,random_2,100,gs://reczilla-results/dataset-splits/splits-v3...,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,20220417_233953,123.379156,12.692332,123.481905
8,UserKNNCF_asymmetric,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40...",MovieTweetingsReader,NaN,full-experiment-knn-146,random_3,100,gs://reczilla-results/dataset-splits/splits-v3...,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,20220417_233953,123.355272,14.126628,123.627794


In [114]:

# how many experiments, and how many samples per experiment?
# print(df_expt.groupby("experiment_name").size().rename("num_rows").reset_index())
print(df_expt.groupby(["alg_name", "dataset_name"]).size().rename("num_rows").reset_index())

                       alg_name               dataset_name  num_rows
0                  CoClustering                AnimeReader       100
1                  CoClustering         BookCrossingReader       100
2                  CoClustering              CiaoDVDReader       100
3                  CoClustering               DatingReader       100
4                  CoClustering             EpinionsReader       100
5                  CoClustering            FilmTrustReader       100
6                  CoClustering               FrappeReader       100
7                  CoClustering   GoogleLocalReviewsReader       100
8                  CoClustering              GowallaReader       100
9                  CoClustering              Jester2Reader       100
10                 CoClustering               LastFMReader       100
11                 CoClustering     MarketBiasAmazonReader       100
12                 CoClustering   MarketBiasModClothReader       100
13                 CoClustering   

In [115]:
# just to be sure, create a column that joins the algorithm and dataset name
df_expt.loc[:, "alg_data_name"] = df_expt[["alg_name", "dataset_name"]].agg('-'.join, axis=1)

/Users/zzzzzz/miniconda3/envs/recsys/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/zzzzzz/miniconda3/envs/recsys/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [116]:
# if there are any rows with no metrics, drop them...
metric_col_list = [c for c in df_expt.columns if "_metric_" in c]

df_expt.loc[:, "all_na_metrics"] = df_expt[metric_col_list].isna().all(axis=1)

# drop cols with all na metrics
df_expt = df_expt.loc[~df_expt["all_na_metrics"], :]

/Users/zzzzzz/miniconda3/envs/recsys/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/zzzzzz/miniconda3/envs/recsys/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [117]:
###### JUST FOR POST-HOC ANALYSIS: treat all UserKNN and ItemKNN algs as the same. this will make it look like there are about 5x more samples for all knn algs than the others
df_expt.loc[df_expt["alg_name"].str.contains("UserKNN"), "alg_name"] = "UserKNN"
df_expt.loc[df_expt["alg_name"].str.contains("ItemKNN"), "alg_name"] = "ItemKNN"


In [118]:
# now find the max and min metric for each alg-dataset combo (just in case there are any min metrics...)
metric_col_list = [c for c in df_expt.columns if "_metric_" in c]

agg_dict = {
    metric: ["min", "max"]
    for metric in metric_col_list
}
agg_dict["time_on_test"] = "median"
agg_dict["time_on_train"] = "median"
agg_dict["time_on_val"] = "median"
agg_dict["time_on_val"] = "size"

df_hpo = df_expt.groupby(["dataset_name", "alg_name"]).agg(agg_dict).reset_index()


In [119]:
new_col_names = ['_'.join(reversed(col)).strip() for col in df_hpo.columns.values]
new_col_names[0] = "dataset_name"
new_col_names[1] = "alg_name"
new_col_names[-1] = "num_samples"


df_hpo.columns = new_col_names
df_hpo.head()

,dataset_name,alg_name,min_test_metric_ARHR_ALL_HITS_cut_1,max_test_metric_ARHR_ALL_HITS_cut_1,min_test_metric_ARHR_ALL_HITS_cut_10,max_test_metric_ARHR_ALL_HITS_cut_10,min_test_metric_ARHR_ALL_HITS_cut_15,max_test_metric_ARHR_ALL_HITS_cut_15,min_test_metric_ARHR_ALL_HITS_cut_2,max_test_metric_ARHR_ALL_HITS_cut_2,...,max_test_metric_USERS_IN_GT_cut_6,min_test_metric_USERS_IN_GT_cut_7,max_test_metric_USERS_IN_GT_cut_7,min_test_metric_USERS_IN_GT_cut_8,max_test_metric_USERS_IN_GT_cut_8,min_test_metric_USERS_IN_GT_cut_9,max_test_metric_USERS_IN_GT_cut_9,median_time_on_test,median_time_on_train,num_samples
0,AnimeReader,CoClustering,0.000000,0.000043,0.000092,0.000410,0.000288,0.000673,0.000007,0.000101,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,255.463675,6201.424713,5
1,AnimeReader,EASE_R_Recommender,0.000000,0.034277,0.000442,0.059715,0.000563,0.062664,0.000079,0.043217,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,296.782406,131.096352,49
2,AnimeReader,GlobalEffects,0.000187,0.000187,0.004560,0.004560,0.005064,0.005064,0.000245,0.000245,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,225.552996,0.704396,1
3,AnimeReader,IALSRecommender,0.015290,0.015290,0.036573,0.036573,0.039374,0.039374,0.021842,0.021842,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,240.805024,3878.538749,1
4,AnimeReader,ItemKNN,0.000000,0.043915,0.000002,0.070494,0.000004,0.073168,0.000000,0.054840,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,273.636642,13.685998,363


In [120]:
df_hpo.to_csv("./performance_meta_dataset.csv")

In [121]:
# how many datasets are there for each alg?
print(df_hpo.groupby(["alg_name"]).size().rename("num_rows").reset_index())

                              alg_name  num_rows
0                         CoClustering        20
1           DELF_EF_RecommenderWrapper         4
2                   EASE_R_Recommender        13
3                        GlobalEffects        21
4                      IALSRecommender        19
5                              ItemKNN        21
6    MatrixFactorization_AsySVD_Cython        15
7       MatrixFactorization_BPR_Cython        21
8   MatrixFactorization_FunkSVD_Cython        20
9          Mult_VAE_RecommenderWrapper        11
10                      NMFRecommender        19
11                  P3alphaRecommender        20
12                  PureSVDRecommender        21
13                  RP3betaRecommender        20
14                              Random        21
15           SLIMElasticNetRecommender        17
16                     SLIM_BPR_Cython        20
17                            SlopeOne        12
18                              TopPop        21
19                  

In [122]:
# ... and how many algs for each dataset?
print(df_hpo.groupby(["dataset_name"]).size().rename("num_rows").reset_index())

                 dataset_name  num_rows
0                 AnimeReader        16
1          BookCrossingReader        15
2               CiaoDVDReader        19
3                DatingReader        13
4              EpinionsReader        16
5             FilmTrustReader        20
6                FrappeReader        20
7               GowallaReader        12
8               Jester2Reader        18
9                LastFMReader        18
10     MarketBiasAmazonReader        19
11   MarketBiasModClothReader        19
12       MovieTweetingsReader        16
13        Movielens100KReader        20
14         Movielens10MReader        17
15          Movielens1MReader        19
16         Movielens20MReader        15
17  MovielensHetrec2011Reader        18
18         NetflixPrizeReader        12
19              RecipesReader        15
20             WikilensReader        20
